In [9]:
import pandas as pd
import numpy as np
import datasets
from inference_utils import *
import json
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import utils
from utils import  get_stats

### read jsonl file

raw_predcitons = []

with open('evalute_outputs/all/raw_outputs.jsonl') as f:
    for line in f:
        raw_predcitons.append(json.loads(line))
        

raw_predcitons = [str(i['generated_text']) for i in raw_predcitons]





In [10]:
model_outputs = extract_predictions(raw_predcitons)

In [11]:
data = datasets.load_dataset('boda/review_evaluation_automatic_labels', 'all')['test']

gold_data = get_gold_labels(data, 'all','chatgpt_ASPECT_score')

In [12]:
assert len(model_outputs) == len(gold_data)

In [15]:
gold_data[0]

{'actionability': 3,
 'grounding_specificity': 5,
 'verifiability': '3',
 'helpfulness': 4}

In [20]:
aspects = [ 'actionability', 'grounding_specificity','verifiability', 'helpfulness']
## convert the list of dictionaries to dictionary of lists

predctions = {aspect:[] for aspect in aspects}
labels = {aspect:[] for aspect in aspects}

for i in range(len(model_outputs)):
    for aspect in aspects:

        l = gold_data[i][aspect]
        p = model_outputs[i][f'{aspect}_label']
        l = str(l)
        p = str(p)
        predctions[aspect].append(p)
        labels[aspect].append(l)

In [26]:
## write the stats into file
with open('evalute_outputs/all/evalute_outputs.txt', 'w') as f:
    for aspect in aspects:
        stat_dict = get_stats(predctions[aspect], labels[aspect], aspect)
        f.write('*' * 20 + f'{aspect}'+ '*' * 20 + '\n')

        for k,v in stat_dict.items():
            ## round the values
            if isinstance(v, float):
                v = round(v, 3)
            f.write(f'{k}: {v}\n')
        f.write('-'*50+'\n')

